In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
from itertools import combinations, permutations
import matplotlib.pyplot as plt
import pickle

In [2]:
import preprocessing
import ModelEvaluator
import shutuba as sh

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
race_resluts = pd.read_pickle("../data/original_data/race_results.pickle")
horse_results = pd.read_pickle("../data/original_data/horse_results.pickle")
return_table = pd.read_pickle("../data/original_data/return_tables.pickle")

In [5]:
with open('le_horse.pickle', 'rb') as web:
  le_horse = pickle.load(web)
with open('le_jockey.pickle', 'rb') as web:
  le_jockey = pickle.load(web)

In [6]:
lgb_clf_danger = pickle.load(open('little_dangerous_pred.pkl', 'rb'))
lgb_clf_safety = pickle.load(open('very_safety.pkl', 'rb'))

In [7]:
p_horse_results = preprocessing.horse_preprocessing(horse_results)
m_race_results = pd.read_pickle("m_race_results.pickle")

c:\keiba\keiba\main\preprocessing.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["着順"] = pd.to_numeric(df["着順"], errors="coerce")
c:\keiba\keiba\main\preprocessing.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["着順"], inplace=True)
c:\keiba\keiba\main\preprocessing.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [41]:
horse_id_list = sh.shutuba_scraping_horse_id("202307011211")

In [42]:
set(horse_id_list) ^ set(p_horse_results[p_horse_results.index.isin(horse_id_list)].index.unique())

set()

In [53]:
popular =  [13,4,1,3,5,7,14,9,11,12,16,15,10,8,2,6]

In [55]:
shutuba = sh.shutuba_scrape(["202305010411"], "2023/02/05")
shutuba["人気"] = popular
p_shutuba = sh.shutuba_preprocessing(shutuba)
m_shutuba = preprocessing.merge_race_with_horse(p_shutuba, p_horse_results)

  0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
d_shutuba = preprocessing.dummy_with_category(le_horse, le_jockey, m_shutuba, m_race_results)

In [57]:
d_shutuba

,枠番,馬番,斤量,人気,course_len,date,horse_id,jockey_id,age,weight,change_weight,n_horses,place,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_allR,賞金_allR,着順/頭数_allR,着順_7R,賞金_7R,着順/頭数_7R,weather_曇,weather_晴,weather_小雨,weather_雨,weather_小雪,weather_雪,race_type_芝,race_type_ダート,race_type_障害,ground_state_良,ground_state_不良,ground_state_稍重,ground_state_重,gender_牡,gender_牝,gender_セ
202305010411,1,1,57,13,16.0,2023-02-05,8733,99,6,464,6,16,5,1.000000,2499.566667,92.622549,3.2,1499.74,72.017974,4.625000,915.693750,67.769754,3.428571,1197.671429,73.187442,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,1,2,58,4,16.0,2023-02-05,9182,84,6,518,10,16,5,4.666667,2228.466667,72.006303,4.2,1842.54,74.718933,4.315789,924.421053,69.811479,3.428571,1686.757143,79.263524,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,2,3,57,1,16.0,2023-02-05,15915,26,5,502,4,16,5,4.000000,1123.333333,70.457516,2.8,1357.16,79.357843,3.166667,815.033333,76.474524,2.571429,1119.314286,80.200657,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,2,4,55,3,16.0,2023-02-05,19486,120,4,406,-6,16,5,4.666667,1320.566667,71.786492,4.2,1478.42,74.849673,3.666667,1348.683333,77.850918,3.666667,1348.683333,77.850918,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
202305010411,3,5,57,5,16.0,2023-02-05,21016,182,4,506,2,16,5,6.333333,1057.866667,59.768519,5.0,1146.92,64.547980,4.000000,932.085714,71.304113,4.000000,932.085714,71.304113,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,3,6,57,7,16.0,2023-02-05,18418,4,4,484,-2,16,5,10.000000,0.000000,40.345861,9.8,1061.84,43.096405,6.111111,1433.644444,63.308658,7.428571,1655.371429,55.804792,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,4,7,58,14,16.0,2023-02-05,13193,67,5,492,8,16,5,7.333333,0.000000,55.046296,6.0,388.40,59.694444,5.894737,634.678947,58.566141,6.000000,871.242857,59.092971,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,4,8,58,9,16.0,2023-02-05,11339,66,6,472,-2,16,5,5.666667,1719.233333,65.695701,4.4,1536.86,73.195199,3.600000,1000.785000,74.077695,4.142857,1244.228571,74.761276,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,5,9,57,11,16.0,2023-02-05,20957,166,4,524,4,16,5,7.666667,878.100000,47.756410,7.2,849.46,50.910256,5.727273,524.300000,57.837995,5.857143,729.614286,59.935897,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010411,5,10,57,12,16.0,2023-02-05,5969,136,7,480,4,16,5,4.333333,675.400000,70.595238,3.2,1112.80,78.318681,2.933333,913.706667,78.662356,4.285714,794.857143,73.402233,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0


In [58]:
d_shutuba_danger = d_shutuba.drop(columns=["date","人気"], axis=1)
d_shutuba_safety = d_shutuba.drop(columns="date", axis=1)

In [59]:
proba_danger = ModelEvaluator.predict_proba(lgb_clf_danger, d_shutuba_danger, train=False, minmax=False)
proba_safety = ModelEvaluator.predict_proba(lgb_clf_safety, d_shutuba_safety, train=False, minmax=False)

[LightGBM] [Warning] Unknown parameter: axis
[LightGBM] [Warning] Unknown parameter: axis


In [60]:
test = shutuba
test["danger_score"] = proba_danger
test["safety_score"] = proba_safety

In [61]:
test

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed:9_level_1,人気,登録,メモ,course_len,race_type,weather,ground_state,date,horse_id,jockey_id,danger_score,safety_score
202305010411,1,1,NaN,プリンスリターン,牡6,57.0,横山和,栗東加用,464(+6),---.-,13,NaN,NaN,1600,芝,晴,良,2023/02/05,2017100661,01140,-0.620496,-0.940271
202305010411,1,2,NaN,ウインカーネリアン,牡6,58.0,三浦,美浦鹿戸,518(+10),---.-,4,NaN,NaN,1600,芝,晴,良,2023/02/05,2017101429,01122,-1.038191,0.483293
202305010411,2,3,NaN,ジャスティンカフェ,牡5,57.0,福永,栗東安田翔,502(+4),---.-,1,NaN,NaN,1600,芝,晴,良,2023/02/05,2018104576,01014,2.580852,3.135553
202305010411,2,4,NaN,ピンハイ,牝4,55.0,坂井,栗東田中克,406(-6),---.-,3,NaN,NaN,1600,芝,晴,良,2023/02/05,2019102772,01163,-0.503960,0.646645
202305010411,3,5,NaN,インダストリア,牡4,57.0,戸崎圭,美浦宮田,506(+2),---.-,5,NaN,NaN,1600,芝,晴,良,2023/02/05,2019105074,05386,-0.156693,0.698506
202305010411,3,6,NaN,マテンロウオリオン,牡4,57.0,横山典,栗東昆,484(-2),---.-,7,NaN,NaN,1600,芝,晴,良,2023/02/05,2019100965,00660,1.017869,-0.077659
202305010411,4,7,NaN,タイムトゥヘヴン,牡5,58.0,大野,美浦戸田,492(+8),---.-,14,NaN,NaN,1600,芝,晴,良,2023/02/05,2018100184,01096,0.513011,-0.936888
202305010411,4,8,NaN,ファルコニア,牡6,58.0,吉田隼,栗東高野,472(-2),---.-,9,NaN,NaN,1600,芝,晴,良,2023/02/05,2017104719,01095,0.472164,-0.148477
202305010411,5,9,NaN,ショウナンマグマ,牡4,57.0,Ｍデムーロ,美浦尾関,524(+4),---.-,11,NaN,NaN,1600,芝,晴,良,2023/02/05,2019104998,05212,0.066651,-0.619025
202305010411,5,10,NaN,ピースワンパラディ,牡7,57.0,菅原明,美浦大竹,480(+4),---.-,12,NaN,NaN,1600,芝,晴,良,2023/02/05,2016103250,01179,0.204913,-0.310793


In [62]:
test[["馬番", "馬名", "人気", "danger_score", "safety_score"]].sort_values("人気")

,馬番,馬名,人気,danger_score,safety_score
202305010411,3,ジャスティンカフェ,1,2.580852,3.135553
202305010411,15,ナミュール,2,0.628072,0.670756
202305010411,4,ピンハイ,3,-0.503960,0.646645
202305010411,2,ウインカーネリアン,4,-1.038191,0.483293
202305010411,5,インダストリア,5,-0.156693,0.698506
202305010411,16,プレサージュリフト,6,0.922726,-0.094757
202305010411,6,マテンロウオリオン,7,1.017869,-0.077659
202305010411,14,エアロロノア,8,-0.347987,0.066180
202305010411,8,ファルコニア,9,0.472164,-0.148477
202305010411,13,サクラトゥジュール,10,-1.269634,-0.614897


In [52]:
test

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed:9_level_1,人気,登録,メモ,course_len,race_type,weather,ground_state,date,horse_id,jockey_id,danger_score,safety_score
202307011211,1,1,NaN,シェイクユアハート,牡3,56.0,岩田康,栗東宮,430(-2),---.-,6,NaN,NaN,2000,芝,晴,良,2023/02/05,2020103101,05203,-1.766972,-1.421744
202307011211,2,2,NaN,フリームファクシ,牡3,56.0,川田,栗東須貝,502(-4),---.-,1,NaN,NaN,2000,芝,晴,良,2023/02/05,2020103661,01088,0.819212,1.254896
202307011211,3,3,NaN,クールミラボー,牡3,56.0,和田竜,栗東寺島,476(-8),---.-,7,NaN,NaN,2000,芝,晴,良,2023/02/05,2020103387,01018,-0.120150,-0.955748
202307011211,4,4,NaN,レミージュ,牝3,54.0,荻野極,栗東松永幹,436(+2),---.-,3,NaN,NaN,2000,芝,晴,良,2023/02/05,2020105642,01160,0.099621,1.177746
202307011211,5,5,NaN,ロゼル,牡3,56.0,松山,美浦大和田,460(0),---.-,4,NaN,NaN,2000,芝,晴,良,2023/02/05,2020106415,01126,1.360261,0.450646
202307011211,6,6,NaN,トーセントラム,牡3,56.0,斎藤,美浦小桧山,440(+2),---.-,8,NaN,NaN,2000,芝,晴,良,2023/02/05,2020102535,01178,-1.237479,-1.224962
202307011211,7,7,NaN,オープンファイア,牡3,56.0,ムルザバエ,栗東斉藤崇,502(-6),---.-,2,NaN,NaN,2000,芝,晴,良,2023/02/05,2020103373,05627,0.893517,0.625222
202307011211,8,8,NaN,ノーブルライジング,牡3,56.0,国分恭,栗東宮本,486(-4),---.-,5,NaN,NaN,2000,芝,晴,良,2023/02/05,2020101697,01124,-0.048011,0.093945
